In [1]:
# prepend parent dir to import from local pcsp
import sys
sys.path.insert(0, '../')

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pcsp
from pcsp import PCSPipeline, ModuleSet, Module, init_args, convert
from pcsp.pipeline import build_graph
from functools import partial
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import sklearn.utils
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from irf import irf_utils, irf_jupyter_utils
from irf.irf_utils import run_iRF
from irf.ensemble import RandomForestClassifierWithWeights
from sklearn.inspection import permutation_importance

In [3]:
# load data
X_train = np.asarray(pd.read_csv("../data/enhancer/01_X_train.csv",error_bad_lines=False).iloc[:,1:])
X_test = np.asarray(pd.read_csv("../data/enhancer/02_X_test.csv",error_bad_lines=False).iloc[:,1:])
y_train = np.asarray(pd.read_csv("../data/enhancer/03_y_train.csv",error_bad_lines=False).iloc[:,1])
y_test = np.asarray(pd.read_csv("../data/enhancer/04_y_test.csv",error_bad_lines=False).iloc[:,1])

# initialize data
np.random.seed(14)
X_train, X_test, y_train, y_test = init_args((X_train, X_test, y_train, y_test), names=['X_train', 'X_test', 'y_train', 'y_test'])

# subsample
subsampling_fns = [partial(sklearn.utils.resample, n_samples=1000, random_state=i) for i in range(3)]

subsampling_set = ModuleSet(name='subsampling', modules=subsampling_fns)
X_trains, y_trains = subsampling_set(X_train, y_train)

modeling_set = ModuleSet(name='modeling', modules=[RandomForestClassifier(n_estimators=50, max_depth=5), GaussianNB()], module_keys=["RF", "NB"])

# model
modeling_set.fit(X_trains, y_trains)
preds = modeling_set.predict(X_test)

# hard metrics
hard_metrics_set = ModuleSet(name='hard_metrics', modules=[accuracy_score, balanced_accuracy_score], module_keys=["Acc", "Bal_Acc"])
hard_metrics = hard_metrics_set.evaluate(preds, y_test)

# permutation importance
feature_importance_set = ModuleSet(name='feature_importance', modules=[permutation_importance])
importances = feature_importance_set.evaluate(modeling_set.out, X_test, y_test)

G = build_graph(importances, draw=True)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '../data/enhancer/01_X_train.csv'

In [4]:
print(hard_metrics)

NameError: name 'hard_metrics' is not defined

In [5]:
print({k:np.argsort(v.importances_mean)[-5:][::-1] for (k, v) in importances.items() if type(v) != pcsp.module_set.ModuleSet})

{(('X_train', 'y_train', 'subsampling_0', 'RF'), 'X_test', 'y_test', 'feature_importance_0'): array([49, 56, 57, 78, 12]), (('X_train', 'y_train', 'subsampling_0', 'NB'), 'X_test', 'y_test', 'feature_importance_0'): array([72, 61, 45, 57, 76]), (('X_train', 'y_train', 'subsampling_1', 'RF'), 'X_test', 'y_test', 'feature_importance_0'): array([47, 68, 20, 13,  6]), (('X_train', 'y_train', 'subsampling_1', 'NB'), 'X_test', 'y_test', 'feature_importance_0'): array([61, 45, 72, 76, 27]), (('X_train', 'y_train', 'subsampling_2', 'RF'), 'X_test', 'y_test', 'feature_importance_0'): array([66, 53,  7, 67, 62]), (('X_train', 'y_train', 'subsampling_2', 'NB'), 'X_test', 'y_test', 'feature_importance_0'): array([45, 61, 72, 27, 23])}
